In [1]:
# Script demonstrating pruning of a conv layer together with a subsequent fully connected layer.
# The Fully connected has no bias.

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
import pdb
import numpy as np
from copy import deepcopy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                       transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                       transform=transform)

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 1000}
if torch.cuda.is_available():
    cuda_kwargs = {'num_workers': 1,
                       'pin_memory': True,
                       'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [4]:
def train(log_interval, model, device, train_loader, optimizer, epoch, alpha =1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data, alpha)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {}, alpha: {}, [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, alpha, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))



def test(model, device, test_loader, alpha=1):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data, alpha)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Alpha {}, Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(alpha,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
# Original Pre-trained model (Trained using mnist.py)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 256, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(256)

        self.fc1 = nn.Linear(256 * 1 * 1, 128, bias =False)
        self.fc2 = nn.Linear(128, 50, bias=False)
        self.fc3 = nn.Linear(50, 10, bias = False)
        

    def forward(self, x, alpha = 1):
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
model = Net().to(device)

In [6]:
# Load weights
# Note that the model loaded was trained with FC layers having no bias
PATH = 'mnist_cnn_nobias.pt'
model_dict= torch.load(PATH, map_location = device)
model.load_state_dict(model_dict)

<All keys matched successfully>

In [7]:
# Test to see if the weights of the pretrained model have been correctly loaded
test(model, device, test_loader)


Test set: Alpha 1, Average loss: 0.1174, Accuracy: 9660/10000 (97%)



In [8]:
class PrunedNet(nn.Module):
    def __init__(self,pretrainedmodel):
        super(PrunedNet, self).__init__()
        self.pretrainedNet = pretrainedmodel
        
        # No need to update weights of the pretrained network        
        for param in self.pretrainedNet.parameters():
            param.requires_grad=False
        # In case the weights of the pretrained network are also desired to be updated then 
        # comment out the above 2 lines of code            
            
        # Branch parallel to conv3 in the original pre-trained network
        # Note that the since, the output of conv3 is connected to a FC layer
        # Therefore fca is also a FC layer. Otherwise if conv3 output was connected to another conv layer
        # Than fca would have been a nn.Conv2d instead of nn.Linear
        self.conva = nn.Conv2d(256, 40, 1, 1, bias = False)
        self.fca = nn.Linear(40, 256, bias = False)
        
        self.bna = nn.BatchNorm2d(40)
        


        # Branch parallel to the fc1 layer in the original pre-trained network
        self.fcc = nn.Linear(128, 64 , bias = False)
        self.fcd = nn.Linear(64, 128, bias = False)
        
    def forward(self, x, alpha=1):
        x = self.pretrainedNet.conv1(x)
        x = self.pretrainedNet.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.pretrainedNet.conv2(x)
        x = self.pretrainedNet.bn2(x)
        x = F.relu(x)
        xorg = F.max_pool2d(x, 2)
        
        
        x1 = self.pretrainedNet.conv3(xorg)

        x2 = self.fca((F.max_pool2d(F.relu(self.bna(self.conva(x1))),2)).squeeze(2).squeeze(2)).unsqueeze(2).unsqueeze(2)
        
        x = alpha * F.max_pool2d(F.relu(self.pretrainedNet.bn3(x1)),2) + (1-alpha) * x2
        
        
        xorg = torch.flatten(x, 1)
                     
        
        x1 = self.pretrainedNet.fc1(xorg)
        
        x2 = self.fcd((F.relu(self.fcc(x1))))

        x = alpha * F.relu(x1) + (1-alpha) * x2

        
        x = self.pretrainedNet.fc2(x)

        x= F.relu(x)
        
        x = self.pretrainedNet.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
pruned_model = PrunedNet(model).to(device)

In [11]:
# Check to see if the original branch(alpha = 1 ) gives the same accuracy.
# If not, then something wrong with the implementation
print('Original accuracy:')
test(model, device, test_loader)
print('Accuracy of pruned_model at alpha = 1:')
print(test(pruned_model, device, test_loader, alpha = 1))

Original accuracy:

Test set: Alpha 1, Average loss: 0.1178, Accuracy: 9658/10000 (97%)

Accuracy of pruned_model at alpha = 1:

Test set: Alpha 1, Average loss: 0.1178, Accuracy: 9658/10000 (97%)

None


In [10]:
# 2 For loops, The outer is for varying the alpha values. The inner loop are the training epochs
# The schedule of how alpha is decayed is something to explore.
# In this case, a simple, linear decay has been used.
optimizer = optim.Adadelta(filter(lambda p: p.requires_grad, pruned_model.parameters()), lr=1.0)
for alpha in np.arange(0.9,-0.05,-0.1):
    scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
    for epoch in range(1, 10 + 1):
        train(10,pruned_model, device, train_loader, optimizer, epoch, alpha)
        test(pruned_model, device, test_loader, alpha)
        scheduler.step()

Train Epoch: 1, alpha: 0.9, [0/60000 (0%)]	Loss: 0.066925
Train Epoch: 1, alpha: 0.9, [640/60000 (1%)]	Loss: 0.051030
Train Epoch: 1, alpha: 0.9, [1280/60000 (2%)]	Loss: 0.188037
Train Epoch: 1, alpha: 0.9, [1920/60000 (3%)]	Loss: 0.056127
Train Epoch: 1, alpha: 0.9, [2560/60000 (4%)]	Loss: 0.043804
Train Epoch: 1, alpha: 0.9, [3200/60000 (5%)]	Loss: 0.131202
Train Epoch: 1, alpha: 0.9, [3840/60000 (6%)]	Loss: 0.028820
Train Epoch: 1, alpha: 0.9, [4480/60000 (7%)]	Loss: 0.078801
Train Epoch: 1, alpha: 0.9, [5120/60000 (9%)]	Loss: 0.178416
Train Epoch: 1, alpha: 0.9, [5760/60000 (10%)]	Loss: 0.174850
Train Epoch: 1, alpha: 0.9, [6400/60000 (11%)]	Loss: 0.057518
Train Epoch: 1, alpha: 0.9, [7040/60000 (12%)]	Loss: 0.067101
Train Epoch: 1, alpha: 0.9, [7680/60000 (13%)]	Loss: 0.075898
Train Epoch: 1, alpha: 0.9, [8320/60000 (14%)]	Loss: 0.179509
Train Epoch: 1, alpha: 0.9, [8960/60000 (15%)]	Loss: 0.059994
Train Epoch: 1, alpha: 0.9, [9600/60000 (16%)]	Loss: 0.064680
Train Epoch: 1, alpha

Train Epoch: 2, alpha: 0.9, [23680/60000 (39%)]	Loss: 0.105808
Train Epoch: 2, alpha: 0.9, [24320/60000 (41%)]	Loss: 0.143939
Train Epoch: 2, alpha: 0.9, [24960/60000 (42%)]	Loss: 0.113059
Train Epoch: 2, alpha: 0.9, [25600/60000 (43%)]	Loss: 0.134374
Train Epoch: 2, alpha: 0.9, [26240/60000 (44%)]	Loss: 0.028841
Train Epoch: 2, alpha: 0.9, [26880/60000 (45%)]	Loss: 0.023445
Train Epoch: 2, alpha: 0.9, [27520/60000 (46%)]	Loss: 0.028530
Train Epoch: 2, alpha: 0.9, [28160/60000 (47%)]	Loss: 0.109318
Train Epoch: 2, alpha: 0.9, [28800/60000 (48%)]	Loss: 0.029241
Train Epoch: 2, alpha: 0.9, [29440/60000 (49%)]	Loss: 0.147850
Train Epoch: 2, alpha: 0.9, [30080/60000 (50%)]	Loss: 0.126919
Train Epoch: 2, alpha: 0.9, [30720/60000 (51%)]	Loss: 0.061151
Train Epoch: 2, alpha: 0.9, [31360/60000 (52%)]	Loss: 0.068631
Train Epoch: 2, alpha: 0.9, [32000/60000 (53%)]	Loss: 0.023437
Train Epoch: 2, alpha: 0.9, [32640/60000 (54%)]	Loss: 0.074445
Train Epoch: 2, alpha: 0.9, [33280/60000 (55%)]	Loss: 0

Train Epoch: 3, alpha: 0.9, [47360/60000 (79%)]	Loss: 0.177832
Train Epoch: 3, alpha: 0.9, [48000/60000 (80%)]	Loss: 0.057115
Train Epoch: 3, alpha: 0.9, [48640/60000 (81%)]	Loss: 0.055936
Train Epoch: 3, alpha: 0.9, [49280/60000 (82%)]	Loss: 0.026623
Train Epoch: 3, alpha: 0.9, [49920/60000 (83%)]	Loss: 0.063140
Train Epoch: 3, alpha: 0.9, [50560/60000 (84%)]	Loss: 0.056899
Train Epoch: 3, alpha: 0.9, [51200/60000 (85%)]	Loss: 0.050694
Train Epoch: 3, alpha: 0.9, [51840/60000 (86%)]	Loss: 0.053309
Train Epoch: 3, alpha: 0.9, [52480/60000 (87%)]	Loss: 0.037675
Train Epoch: 3, alpha: 0.9, [53120/60000 (88%)]	Loss: 0.025574
Train Epoch: 3, alpha: 0.9, [53760/60000 (90%)]	Loss: 0.016606
Train Epoch: 3, alpha: 0.9, [54400/60000 (91%)]	Loss: 0.063599
Train Epoch: 3, alpha: 0.9, [55040/60000 (92%)]	Loss: 0.096004
Train Epoch: 3, alpha: 0.9, [55680/60000 (93%)]	Loss: 0.057250
Train Epoch: 3, alpha: 0.9, [56320/60000 (94%)]	Loss: 0.161570
Train Epoch: 3, alpha: 0.9, [56960/60000 (95%)]	Loss: 0

Train Epoch: 5, alpha: 0.9, [9600/60000 (16%)]	Loss: 0.061029
Train Epoch: 5, alpha: 0.9, [10240/60000 (17%)]	Loss: 0.107569
Train Epoch: 5, alpha: 0.9, [10880/60000 (18%)]	Loss: 0.052936
Train Epoch: 5, alpha: 0.9, [11520/60000 (19%)]	Loss: 0.184031
Train Epoch: 5, alpha: 0.9, [12160/60000 (20%)]	Loss: 0.022468
Train Epoch: 5, alpha: 0.9, [12800/60000 (21%)]	Loss: 0.029060
Train Epoch: 5, alpha: 0.9, [13440/60000 (22%)]	Loss: 0.006937
Train Epoch: 5, alpha: 0.9, [14080/60000 (23%)]	Loss: 0.070066
Train Epoch: 5, alpha: 0.9, [14720/60000 (25%)]	Loss: 0.041073
Train Epoch: 5, alpha: 0.9, [15360/60000 (26%)]	Loss: 0.114320
Train Epoch: 5, alpha: 0.9, [16000/60000 (27%)]	Loss: 0.034775
Train Epoch: 5, alpha: 0.9, [16640/60000 (28%)]	Loss: 0.045369
Train Epoch: 5, alpha: 0.9, [17280/60000 (29%)]	Loss: 0.012282
Train Epoch: 5, alpha: 0.9, [17920/60000 (30%)]	Loss: 0.018200
Train Epoch: 5, alpha: 0.9, [18560/60000 (31%)]	Loss: 0.074648
Train Epoch: 5, alpha: 0.9, [19200/60000 (32%)]	Loss: 0.

Train Epoch: 6, alpha: 0.9, [32640/60000 (54%)]	Loss: 0.123902
Train Epoch: 6, alpha: 0.9, [33280/60000 (55%)]	Loss: 0.046367
Train Epoch: 6, alpha: 0.9, [33920/60000 (57%)]	Loss: 0.108947
Train Epoch: 6, alpha: 0.9, [34560/60000 (58%)]	Loss: 0.146573
Train Epoch: 6, alpha: 0.9, [35200/60000 (59%)]	Loss: 0.069120
Train Epoch: 6, alpha: 0.9, [35840/60000 (60%)]	Loss: 0.021959
Train Epoch: 6, alpha: 0.9, [36480/60000 (61%)]	Loss: 0.051114
Train Epoch: 6, alpha: 0.9, [37120/60000 (62%)]	Loss: 0.124331
Train Epoch: 6, alpha: 0.9, [37760/60000 (63%)]	Loss: 0.030800
Train Epoch: 6, alpha: 0.9, [38400/60000 (64%)]	Loss: 0.021745
Train Epoch: 6, alpha: 0.9, [39040/60000 (65%)]	Loss: 0.075407
Train Epoch: 6, alpha: 0.9, [39680/60000 (66%)]	Loss: 0.048712
Train Epoch: 6, alpha: 0.9, [40320/60000 (67%)]	Loss: 0.028022
Train Epoch: 6, alpha: 0.9, [40960/60000 (68%)]	Loss: 0.079428
Train Epoch: 6, alpha: 0.9, [41600/60000 (69%)]	Loss: 0.076928
Train Epoch: 6, alpha: 0.9, [42240/60000 (70%)]	Loss: 0

...

Train Epoch: 5, alpha: 2.220446049250313e-16, [55680/60000 (93%)]	Loss: 0.144112
Train Epoch: 5, alpha: 2.220446049250313e-16, [56320/60000 (94%)]	Loss: 0.161331
Train Epoch: 5, alpha: 2.220446049250313e-16, [56960/60000 (95%)]	Loss: 0.136132
Train Epoch: 5, alpha: 2.220446049250313e-16, [57600/60000 (96%)]	Loss: 0.058519
Train Epoch: 5, alpha: 2.220446049250313e-16, [58240/60000 (97%)]	Loss: 0.196802
Train Epoch: 5, alpha: 2.220446049250313e-16, [58880/60000 (98%)]	Loss: 0.228647
Train Epoch: 5, alpha: 2.220446049250313e-16, [59520/60000 (99%)]	Loss: 0.353232

Test set: Alpha 2.220446049250313e-16, Average loss: 0.1922, Accuracy: 9442/10000 (94%)

Train Epoch: 6, alpha: 2.220446049250313e-16, [0/60000 (0%)]	Loss: 0.368035
Train Epoch: 6, alpha: 2.220446049250313e-16, [640/60000 (1%)]	Loss: 0.049930
Train Epoch: 6, alpha: 2.220446049250313e-16, [1280/60000 (2%)]	Loss: 0.117577
Train Epoch: 6, alpha: 2.220446049250313e-16, [1920/60000 (3%)]	Loss: 0.208145
Train Epoch: 6, alpha: 2.220446


Test set: Alpha 2.220446049250313e-16, Average loss: 0.1881, Accuracy: 9472/10000 (95%)

Train Epoch: 7, alpha: 2.220446049250313e-16, [0/60000 (0%)]	Loss: 0.431004
Train Epoch: 7, alpha: 2.220446049250313e-16, [640/60000 (1%)]	Loss: 0.146487
Train Epoch: 7, alpha: 2.220446049250313e-16, [1280/60000 (2%)]	Loss: 0.514191
Train Epoch: 7, alpha: 2.220446049250313e-16, [1920/60000 (3%)]	Loss: 0.154460
Train Epoch: 7, alpha: 2.220446049250313e-16, [2560/60000 (4%)]	Loss: 0.104184
Train Epoch: 7, alpha: 2.220446049250313e-16, [3200/60000 (5%)]	Loss: 0.108812
Train Epoch: 7, alpha: 2.220446049250313e-16, [3840/60000 (6%)]	Loss: 0.169901
Train Epoch: 7, alpha: 2.220446049250313e-16, [4480/60000 (7%)]	Loss: 0.090018
Train Epoch: 7, alpha: 2.220446049250313e-16, [5120/60000 (9%)]	Loss: 0.302265
Train Epoch: 7, alpha: 2.220446049250313e-16, [5760/60000 (10%)]	Loss: 0.235297
Train Epoch: 7, alpha: 2.220446049250313e-16, [6400/60000 (11%)]	Loss: 0.027916
Train Epoch: 7, alpha: 2.220446049250313e-1

Train Epoch: 8, alpha: 2.220446049250313e-16, [5120/60000 (9%)]	Loss: 0.270374
Train Epoch: 8, alpha: 2.220446049250313e-16, [5760/60000 (10%)]	Loss: 0.201466
Train Epoch: 8, alpha: 2.220446049250313e-16, [6400/60000 (11%)]	Loss: 0.047235
Train Epoch: 8, alpha: 2.220446049250313e-16, [7040/60000 (12%)]	Loss: 0.112841
Train Epoch: 8, alpha: 2.220446049250313e-16, [7680/60000 (13%)]	Loss: 0.187850
Train Epoch: 8, alpha: 2.220446049250313e-16, [8320/60000 (14%)]	Loss: 0.169501
Train Epoch: 8, alpha: 2.220446049250313e-16, [8960/60000 (15%)]	Loss: 0.038415
Train Epoch: 8, alpha: 2.220446049250313e-16, [9600/60000 (16%)]	Loss: 0.328772
Train Epoch: 8, alpha: 2.220446049250313e-16, [10240/60000 (17%)]	Loss: 0.116566
Train Epoch: 8, alpha: 2.220446049250313e-16, [10880/60000 (18%)]	Loss: 0.254572
Train Epoch: 8, alpha: 2.220446049250313e-16, [11520/60000 (19%)]	Loss: 0.147873
Train Epoch: 8, alpha: 2.220446049250313e-16, [12160/60000 (20%)]	Loss: 0.285294
Train Epoch: 8, alpha: 2.220446049250

Train Epoch: 9, alpha: 2.220446049250313e-16, [9600/60000 (16%)]	Loss: 0.364789
Train Epoch: 9, alpha: 2.220446049250313e-16, [10240/60000 (17%)]	Loss: 0.292764
Train Epoch: 9, alpha: 2.220446049250313e-16, [10880/60000 (18%)]	Loss: 0.131481
Train Epoch: 9, alpha: 2.220446049250313e-16, [11520/60000 (19%)]	Loss: 0.059926
Train Epoch: 9, alpha: 2.220446049250313e-16, [12160/60000 (20%)]	Loss: 0.020204
Train Epoch: 9, alpha: 2.220446049250313e-16, [12800/60000 (21%)]	Loss: 0.308804
Train Epoch: 9, alpha: 2.220446049250313e-16, [13440/60000 (22%)]	Loss: 0.400592
Train Epoch: 9, alpha: 2.220446049250313e-16, [14080/60000 (23%)]	Loss: 0.132377
Train Epoch: 9, alpha: 2.220446049250313e-16, [14720/60000 (25%)]	Loss: 0.186647
Train Epoch: 9, alpha: 2.220446049250313e-16, [15360/60000 (26%)]	Loss: 0.320600
Train Epoch: 9, alpha: 2.220446049250313e-16, [16000/60000 (27%)]	Loss: 0.179181
Train Epoch: 9, alpha: 2.220446049250313e-16, [16640/60000 (28%)]	Loss: 0.401602
Train Epoch: 9, alpha: 2.2204

Train Epoch: 10, alpha: 2.220446049250313e-16, [14720/60000 (25%)]	Loss: 0.135558
Train Epoch: 10, alpha: 2.220446049250313e-16, [15360/60000 (26%)]	Loss: 0.220546
Train Epoch: 10, alpha: 2.220446049250313e-16, [16000/60000 (27%)]	Loss: 0.129206
Train Epoch: 10, alpha: 2.220446049250313e-16, [16640/60000 (28%)]	Loss: 0.160446
Train Epoch: 10, alpha: 2.220446049250313e-16, [17280/60000 (29%)]	Loss: 0.074302
Train Epoch: 10, alpha: 2.220446049250313e-16, [17920/60000 (30%)]	Loss: 0.200026
Train Epoch: 10, alpha: 2.220446049250313e-16, [18560/60000 (31%)]	Loss: 0.120756
Train Epoch: 10, alpha: 2.220446049250313e-16, [19200/60000 (32%)]	Loss: 0.215223
Train Epoch: 10, alpha: 2.220446049250313e-16, [19840/60000 (33%)]	Loss: 0.057913
Train Epoch: 10, alpha: 2.220446049250313e-16, [20480/60000 (34%)]	Loss: 0.157240
Train Epoch: 10, alpha: 2.220446049250313e-16, [21120/60000 (35%)]	Loss: 0.114997
Train Epoch: 10, alpha: 2.220446049250313e-16, [21760/60000 (36%)]	Loss: 0.110038
Train Epoch: 10,

In [12]:
print(test(pruned_model, device, test_loader, 0 ))
print(test(pruned_model, device, test_loader, 1 ))


Test set: Alpha 0, Average loss: 0.1769, Accuracy: 9482/10000 (95%)

None

Test set: Alpha 1, Average loss: 0.1178, Accuracy: 9658/10000 (97%)

None


In [15]:
# Final pruned model after doing the appropriate matrix multiplications (for FC layers)
# or tensor product for (convolution layers)
# for tensor products see: https://pytorch.org/docs/stable/generated/torch.einsum.html

# After pruning the conv3 has only 40 filters instead of the original 256
# Also the FC1 output has now only 64 neurons instead of 128

# Note that at the point where 2 consecutive layers are being pruned, 3 matrix multiplications need to be done.
# This can also be thought as 2 matrix multiplications done 2 times.
# TODO: Implement bias when there are consecutive layers being pruned
class SmallerNet(nn.Module):
    def __init__(self):
        super(SmallerNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, 3, 1, bias = False)
        self.conv2 = nn.Conv2d(1, 2, 3, 1, bias = False)
        self.conv3 = nn.Conv2d(2, 40, 3, 1, bias = False)
        
        self.bn1 = nn.BatchNorm2d(1)
        self.bn2 = nn.BatchNorm2d(2)
        self.bn3 = nn.BatchNorm2d(40)

        self.fc1 = nn.Linear(40 * 1 * 1, 64, bias =False)
        self.fc2 = nn.Linear(64, 50, bias=False)
        self.fc3 = nn.Linear(50, 10, bias = False)
        

    def forward(self, x, alpha = 1):
        # Note that alpha is not being used here
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x= F.relu(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
smaller_model = SmallerNet().to(device)

In [16]:
# Since the model is randomly initilaized, the accuracy is low
test(smaller_model, device, test_loader, 0 )


Test set: Alpha 0, Average loss: 2.3036, Accuracy: 967/10000 (10%)



In [19]:
# The pretrained weights are loaded after the appropriate matrix/tensor multiplications to get the new weight matrix 

smaller_model_dict= deepcopy(model.state_dict())

pmd=pruned_model.state_dict()
print(pmd["conva.weight"].shape)
# print(pmd["convb.weight"].shape)
print(pmd['pretrainedNet.conv3.weight'].shape)
x = pmd["conva.weight"]
y = pmd['pretrainedNet.conv3.weight']
# 4 dimensional Tensor multplications in the next steps
# 2 alternate ways to do this. Both should yield the same result
out1 = torch.einsum("abcd, befg -> aefg",x,y)
out2 = torch.einsum("abcd, eafg -> ebcd",y,x)

smaller_model_dict["conv3.weight"] = out1
smaller_model_dict["bn3.weight"] = pmd["bna.weight"]
smaller_model_dict["bn3.bias"] = pmd["bna.bias"]
smaller_model_dict["bn3.running_mean"] = pmd["bna.running_mean"]
smaller_model_dict["bn3.running_var"]  = pmd["bna.running_var"]
smaller_model_dict["bn3.num_batches_tracked"] = pmd["bna.num_batches_tracked"]
smaller_model_dict["bn3.weight"] = pmd["bna.weight"]
# Note the 3 matrix multiplications in the next step
smaller_model_dict["fc1.weight"] = pmd["fcc.weight"] @ pmd["pretrainedNet.fc1.weight"] @ pmd["fca.weight"]
smaller_model_dict["fc2.weight"] = pmd["pretrainedNet.fc2.weight"] @ pmd["fcd.weight"]
smaller_model.load_state_dict(smaller_model_dict)

torch.Size([40, 256, 1, 1])
torch.Size([256, 2, 3, 3])


<All keys matched successfully>

In [20]:
test(smaller_model, device, test_loader, 0 )


Test set: Alpha 0, Average loss: 0.1769, Accuracy: 9482/10000 (95%)

